In [1]:
pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in c:\users\strelokask\anaconda3\lib\site-packages (1.12.6)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [3]:
SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'task1-read.json'

In [4]:
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=credentials)

In [5]:
results = service.files().list(fields="files(id, parents, name, mimeType)").execute()
all_folders = [];
for res in results['files']:
    if res['mimeType'] == 'application/vnd.google-apps.folder':
        dict={}
        dict['folder_id'] = res['id']
        dict['folder_name'] = res['name']
        dict['images'] = []
        for obj in results['files']:
            if obj['mimeType'].startswith('image/') and obj['parents'] == [res['id']]:
                dict['images'].append(obj['name'])
        all_folders.append(dict)

In [6]:
def find_full_path(items, folder_id, res):
    for item in items:
        if folder_id == item['id']:
            if "parents" in item:
                return find_full_path(items, item['parents'][0], item['name']+'/'+ res )
            return item['name']+'/'+ res;
    return res

In [7]:
full_path_image = []
full_path_empty_folder = []
for f in all_folders:
    if len(f['images']) > 0:
        full_path = find_full_path(results['files'], f['folder_id'], '')
        for img in f['images']:
            full_path_image.append(['gs://project/' + full_path + img + ', ' + f['folder_name']])
    else:
        path = find_full_path(results['files'], f['folder_id'], '')
        full_path_empty_folder.append([path[:-1]])

['gs://project/List1/Sphera Bold/women/Alpha-W-6/20.jpg, Alpha-W-6']


In [8]:
pip install oauth2client

In [17]:
import httplib2 
from oauth2client.service_account import ServiceAccountCredentials
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [10]:
SCOPES_W = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE_W = 'task1-write.json'
spreadsheet_id = '1rcyTVLcXprdkzK6mbbl-SGKxnL75dOpEoA3XfNPQEx8'

In [11]:
w_credentials = ServiceAccountCredentials.from_json_keyfile_name(
        SERVICE_ACCOUNT_FILE_W, scopes=SCOPES_W)
httpAuth = w_credentials.authorize(httplib2.Http())
w_service = build('sheets', 'v4', http = httpAuth)

In [12]:
spreadsheet = w_service.spreadsheets().get(spreadsheetId = spreadsheet_id).execute()
sheetList = spreadsheet.get('sheets')
sheet_name, sheet_id = [], []
for sheet in sheetList:
    sheet_name.append(sheet['properties']['title'])
    sheet_id.append(sheet['properties']['sheetId'])

In [18]:
range_sheet = str(sheet_name[1]) + '!A1:A'+str(len(full_path_image))
range_sheet_empty_folder = str(sheet_name[2]) + '!A1:A'+str(len(full_path_empty_folder))
values = w_service.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheet_id,
                                                       body = {'valueInputOption':'USER_ENTERED',
                                                               'data' : [{'range': range_sheet,
                                                                          'majorDimension':'ROWS',
                                                                          'values': full_path_image
                                                                          },
                                                                         {'range': range_sheet_empty_folder,
                                                                          'majorDimension':'ROWS',
                                                                          'values': full_path_empty_folder
                                                                          }]
                                                              }
                                                      ).execute()

In [27]:
ranges = [(name + '!A2:A2') for name in sheet_name]
print(ranges)
size = [] 
result = w_service.spreadsheets().get(spreadsheetId = spreadsheet_id, 
                                     ranges = ranges, includeGridData = True).execute()
for i in range(len(ranges)):
    size.append(result['sheets'][i]['data'][0]['columnMetadata'][0])
    print(result['sheets'][i]['data'][0]['columnMetadata'][0])
#pp.pprint(result['sheets'])

['Output example!A2:A2', 'Labels!A2:A2', 'Empty folders!A2:A2']
{'pixelSize': 483}
{'pixelSize': 516}
{'pixelSize': 833}


In [28]:
result = w_service.spreadsheets().batchUpdate(spreadsheetId = spreadsheet_id,
                                                       body = {'requests':{
                                                                    "updateDimensionProperties": {
                                                                        "range": {
                                                                            "sheetId": sheet_id[1],
                                                                            "dimension": "COLUMNS",
                                                                            "startIndex": 0,
                                                                            "endIndex": 1
                                                                        },
                                                                        "properties": {
                                                                          "pixelSize": size[1]['pixelSize']
                                                                        },
                                                                        "fields": "pixelSize"
                                                                      }
                                                                    }
                                                              }
                                                      ).execute()

In [16]:
result = w_service.spreadsheets().batchUpdate(spreadsheetId = spreadsheet_id,
                                                       body = {'requests':{
                                                                    "updateDimensionProperties": {
                                                                        "range": {
                                                                            "sheetId": sheet_id[2],
                                                                            "dimension": "COLUMNS",
                                                                            "startIndex": 0,
                                                                            "endIndex": 1
                                                                        },
                                                                        "properties": {
                                                                          "pixelSize": size[2]['pixelSize']
                                                                        },
                                                                        "fields": "pixelSize"
                                                                      }
                                                                    }
                                                              }
                                                      ).execute()